In [1]:
%matplotlib inline

from utils import FMA
import numpy as np
import IPython.display as ipd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn import utils as skl_utils
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from score import Audio

In [2]:
fma = FMA()
tracks = fma.load('data/fma_metadata/tracks.csv')
genres = fma.load('data/fma_metadata/genres.csv')
features = fma.load('data/fma_metadata/features.csv')
tracks.shape, genres.shape, features.shape

((106574, 52), (163, 4), (106574, 518))

In [4]:
medium = tracks['set', 'subset'] <= 'medium'
ipd.display(tracks.loc[medium].head())
ipd.display(features.loc[medium].head())

album                                                          \
         comments        date_created date_released engineer favorites id   
track_id                                                                    
2               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
3               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
5               0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   
10              0 2008-11-26 01:45:08    2008-02-06      NaN         4  6   
134             0 2008-11-26 01:44:45    2009-01-05      NaN         4  1   

                                            ...       track           \
         information listens producer tags  ... information interest   
track_id                                    ...                        
2            <p></p>    6073      NaN   []  ...         NaN     4656   
3            <p></p>    6073      NaN   []  ...         NaN     1470   
5            <p></p>    6073      NaN   []  ...         NaN     1933   
10               NaN   47632      NaN   []  ...         NaN    54881   
134          <p></p>    6073      NaN   []  ...         NaN     1126   

                                                                           \
         language_code                                            license   
track_id                                                                    
2                   en  Attribution-NonCommercial-ShareAlike 3.0 Inter...   
3                   en  Attribution-NonCommercial-ShareAlike 3.0 Inter...   
5                   en  Attribution-NonCommercial-ShareAlike 3.0 Inter...   
10                  en  Attribution-NonCommercial-NoDerivatives (aka M...   
134                 en  Attribution-NonCommercial-ShareAlike 3.0 Inter...   

                                                               
         listens lyricist number publisher tags         title  
track_id                                                       
2           1293      NaN      3       NaN   []          Food  
3            514      NaN      4       NaN   []  Electric Ave  
5           1151      NaN      6       NaN   []    This World  
10         50135      NaN      1       NaN   []       Freeway  
134          943      NaN      5       NaN   []  Street Music  

[5 rows x 52 columns]

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
2             7.180653  5.230309  0.249321  1.347620  1.482478  0.531371   
3             1.888963  0.760539  0.345297  2.295201  1.654031  0.067592   
5             0.527563 -0.077654 -0.279610  0.685883  1.937570  0.880839   
10            3.702245 -0.291193  2.196742 -0.234449  1.367364  0.998411   
134           0.918445  0.674147  0.577818  1.281117  0.933746  0.078177   

feature                                             ...   tonnetz            \
statistics                                          ...       std             
number            07        08        09        10  ...        04        05   
track_id                                            ...                       
2           1.481593  2.691455  0.866868  1.341231  ...  0.054125  0.012226   
3           1.366848  1.054094  0.108103  0.619185  ...  0.063831  0.014212   
5          -0.923192 -0.927232  0.666617  1.038546  ...  0.040730  0.012691   
10          1.770694  1.604566  0.521217  1.982386  ...  0.074358  0.017952   
134         1.199204 -0.175223  0.925482  1.438509  ...  0.058766  0.016322   

feature                     zcr                                               \
statistics             kurtosis       max      mean    median  min      skew   
number            06         01        01        01        01   01        01   
track_id                                                                       
2           0.012111   5.758890  0.459473  0.085629  0.071289  0.0  2.089872   
3           0.017740   2.824694  0.466309  0.084578  0.063965  0.0  1.716724   
5           0.014759   6.808415  0.375000  0.053114  0.041504  0.0  2.193303   
10          0.013921  21.434212  0.452148  0.077515  0.071777  0.0  3.542325   
134         0.015819   4.731087  0.419434  0.064370  0.050781  0.0  1.806106   

feature               
statistics       std  
number            01  
track_id              
2           0.061448  
3           0.069330  
5           0.044861  
10          0.040800  
134         0.054623  

[5 rows x 518 columns]

In [32]:
size = 'medium'
top_genres = fma.top_genres(size=size, show=True)
(X_train, Y_train), (X_val, Y_val), (X_test, Y_test) = fma.train_data(size)

Genres               |  Count
----------------------------
Hip-Hop              | 	2201
Pop                  | 	1186
Rock                 | 	7103
Folk                 | 	1519
Experimental         | 	2251
Jazz                 | 	384
Electronic           | 	6314
International        | 	1018
Soul-RnB             | 	154
Blues                | 	74
Spoken               | 	118
Country              | 	178
Classical            | 	619
Old-Time / Historic  | 	510
Instrumental         | 	1350
Easy Listening       | 	21


In [33]:
ssr = StandardScaler(copy=False)
ssr.fit(X_train)

X_train = ssr.transform(X_train)
X_val = ssr.transform(X_val)
X_test = ssr.transform(X_test)

In [34]:
ler = LabelEncoder()
ler.fit(Y_train)

Y_train = ler.transform(Y_train)
Y_val = ler.transform(Y_val)
Y_test = ler.transform(Y_test)

In [35]:
X_train, Y_train = skl_utils.shuffle(X_train, Y_train)

## Read Audio MFCC

In [45]:
audio = Audio(r"D:\CODE\Project\Music_score\src\data\music\download\4n9Ie3ac6BM.mp3", duration=60)
_, _, mfcc_stats = audio.get_mfcc()
stats = np.array(mfcc_stats.to_array()).reshape(1, -1)
stats = ssr.transform(stats)

## Predicts

### Support Vector Classifier, SVC (SVM)

In [46]:
clf = SVC()
clf.fit(X_train, Y_train)

SVC()

In [47]:
score = clf.score(X_test, Y_test)
print(f"Accuracy: {score:.2%}")

Accuracy: 60.98%


In [48]:
pred = clf.predict(stats)
pred = ler.inverse_transform(pred)
print(pred)

['Experimental']


### Logistic Regression, LR

In [40]:
lr = LogisticRegression(C=0.025, solver='saga', max_iter=100)
lr.fit(X_train, Y_train)

d:\CODE\Project\Music_score\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=0.025, solver='saga')

In [42]:
score = lr.score(X_test, Y_test)
print(f"Accuracy: {score:.2%}")

Accuracy: 58.30%


In [49]:
pred = lr.predict(stats)
pred = ler.inverse_transform(pred)
print(pred)

['Folk']


In [52]:
pps = lr.predict_proba(stats)
for i, pp in enumerate(pps[0]):
    class_name = ler.classes_[i]
    print(f"{class_name.ljust(20, " ")}\t: {pp:.2%}", end = "  <<\n" if class_name == pred else "\n")

Blues               	: 0.00%
Classical           	: 0.00%
Country             	: 0.00%
Easy Listening      	: 0.00%
Electronic          	: 8.23%
Experimental        	: 1.28%
Folk                	: 76.51%  <<
Hip-Hop             	: 0.03%
Instrumental        	: 0.83%
International       	: 0.00%
Jazz                	: 0.00%
Old-Time / Historic 	: 12.30%
Pop                 	: 0.59%
Rock                	: 0.02%
Soul-RnB            	: 0.00%
Spoken              	: 0.20%
